In [1]:
import pandas as pd
import syft as sy
import numpy as np
sy.logger.remove()

### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into usa... done!


In [3]:
ca.datasets

Idx,Name,Description,Assets,Id
[0],Canada Trade Data - First few rows4,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",27766675-be5c-4365-87e3-ee03cf9744aa


In [4]:
it.datasets

Idx,Name,Description,Assets,Id
[0],USA Trade Data - First few rows,A collection of reports from USA's statistics bureau about how much it thinks it imports and exports from other countries.,"[""USA Trade""] -> Tensor",93ffb90b-8d87-4933-bd11-4650a022d26b


In [5]:
ca.privacy_budget

200.0

In [6]:
it.privacy_budget

200.0

In [7]:
ca_data = ca.datasets[-1]['Canada Trade']
it_data = it.datasets[-1]['USA Trade']

In [16]:
ca_data

<TensorWrappedSingleEntityPhiTensorPointer -> canada:ecc3e2c8a7534f3d930b89aff6592a00>

In [17]:
it_data

<TensorWrappedSingleEntityPhiTensorPointer -> usa:4f85f2ab66e54415b1d45ef786e78171>

In [18]:
a = ca_data + it_data

In [11]:
result = a.publish(sigma=3)

In [12]:
result.get()

array([ -2,   0,  79,  62,  34, 662,   4,   4,  -5,   1], dtype=int32)

In [13]:
b = ca_data > it_data

((ShareTensor(child=[ True  True  True False  True False  True False False False]), ShareTensor(child=[ True  True  True  True False False False False  True False]), ShareTensor(child=[False False  True False  True False False  True  True False])), (ShareTensor(child=[False  True False  True  True False False False False  True]), ShareTensor(child=[ True  True False False False False False False False False]), ShareTensor(child=[ True  True False False  True False  True  True  True False])), (ShareTensor(child=[ True  True  True  True  True  True  True False  True  True]), ShareTensor(child=[ True False False  True False  True  True False False  True]), ShareTensor(child=[ True False False  True False  True  True False False  True])), (ShareTensor(child=[ True  True  True False False  True False  True  True False]), ShareTensor(child=[False False False False  True False False False False  True]), ShareTensor(child=[False  True False False  True  True  True False  True False])), (ShareT

In [14]:
public_result = b.publish(sigma=2)

In [15]:
public_result.get()

Exception: One of the shares doesn't exist. This probably means the SMPC computation isn't yet complete. Try again in a moment or call .block.reconstruct()instead to block until the SMPC operation is complete which creates this variable.

In [12]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

In [13]:
self = ca_data
other = it_data

In [15]:
parties = [self.client, other.client]
self_mpc = MPCTensor(secret=self, shape=self.public_shape, parties=parties)
other_mpc = MPCTensor(
    secret=other, shape=other.public_shape, parties=parties
)

In [16]:
(
    self_shrs,
    other_shrs,
    kwargs,
    ring_size,
    parties,
    parties_info,
    self_shape,
    other_shape,
) = MPCTensor.prepare_arguments_and_run_checks(self_mpc, other_mpc)

In [21]:
out = self_shrs[0] + other_shrs[0]

In [22]:
out2 = self_shrs[1] + other_shrs[1]

In [28]:
# Step 6: Execute the SMPC operation
# TODO: some complex hooking logic on ShareTensor means we're passing in 'a' twice
res_shares = [a.__add__(b, **kwargs) for a, b in zip(self_shrs, other_shrs)]

In [12]:
result = ca_data + it_data

A:<TensorPointer -> lucid_bostrom:2a873523174947d4837e2a816c83e2b2>
B:<TensorPointer -> lucid_bostrom:c26bfa8936f948cd8829288b56e519d4>


In [19]:
public_result.block_with_timeout(3).get()

TypeError: block_with_timeout() got an unexpected keyword argument 'secs_per_poll'

In [16]:
ca.privacy_budget

41.592815128439895

In [17]:
it.privacy_budget

41.592815128439895

In [18]:
public_result.get()

array([ 0,  1, -1, -2, -1, -3, -7, -2,  0, -2], dtype=int32)